## Pure Python

In [1]:
#Uses python3

import numpy as np
import random as rnd
import time
import pandas as pd
import sys
import matplotlib.pyplot as plt

In [2]:
def kron(i,j):
    """Kroneker's symbol"""
    if i==j: return 1
    else: return 0

def coord(site):
    """get coordinate i of vector"""
    x = site // L
    y = site - x*L
    return (x,y)

def get(i):
    """fixin' boundary"""
    if i<0: return i
    else: return i % L
    
def get_neigh():
    """get neighbour's arr"""
    s = np.arange(L**2).reshape(L,L)
    nei = []
    for site in range(L*L):
        i,j = coord(site)
        nei += [s[get(i-1),get(j)],s[get(i),get(j+1)],s[get(i+1),get(j)],s[get(i),get(j-1)]]
    return np.array(nei, dtype=np.int32).reshape(L*L,4)

#################################################################

def gen_state():
    """generate random start state with lenght L*L and q components"""
    state = np.random.randint(0, q, L*L, dtype=np.int32)
    return state

################################################################################

def calc_e(state):
    s = state.reshape(L,L)
    e = 0
    for i in range(-1,L-1):
        for j in range(-1,L-1):
            e += kron(s[i,j], s[i+1,j]) # right neighbour
            e += kron(s[i,j], s[i,j+1]) # down neighbour
    return -e     # e = -J*qi*qj

def calc_m(state):
    s = state.reshape(L,L)
    m_vect = np.array([np.count_nonzero(s == i) for i in range(q)])
    return (max(m_vect)*q/L**2-1)/(q-1)  #Numerical revision of the ... two-dimensional 4-state Potts model (15)

################################################################################

def model_p4(T,N_avg=10,N_mc=10,Relax=10):
    """Моделируем АТ"""
    E, M = [], []

    state = gen_state()
    nei = get_neigh()
    
    #relax $Relax times be4 AVG
    for __ in range(Relax):
            mc_step(state, nei, T)
    #AVG every $N_mc steps
    for _ in range(N_avg):
        for __ in range(N_mc):
            mc_step(state, nei, T)
        E += [calc_e(state)]
        M += [calc_m(state)]
    
    return E, M

## Cython

In [3]:
import cython
%load_ext Cython

In [140]:
%%cython -a
import cython
import numpy as np
cimport numpy as cnp

from numpy cimport npy_intp, npy_longdouble

from libc.math cimport exp
from cpython cimport bool


@cython.cdivision(True)
@cython.boundscheck(False)
@cython.wraparound(False)
cdef int calc_dE_2(npy_intp site, int new_val,
                   cnp.int32_t[::1] spins,          # the field
                   cnp.int32_t[:, :] neighbors):    # associations: neighbors[site, :] are local neighb sites
    """Calculate dE: the energy change for spins[site] -> new_val."""
    
    cdef:
        int old_val = spins[site]
        int this_val
        int e0 = 0, e1 = 0
        npy_intp site1
           
    for j in range(4):    # FIXME: named constant D=2, DD = 2*d
        site1 = neighbors[site, j]
        this_val = spins[site1]
        if this_val == old_val:
            e1 += 1
        elif this_val == new_val:
            e0 += 1
        else:
            continue
    
    return e1 - e0

@cython.cdivision(True)
@cython.boundscheck(False)
@cython.wraparound(False)
cdef bint mc_choice(int dE, double T, npy_longdouble uni):
    """принимаем или не принимаем переворот спина?"""
    cdef double r
    r = exp(-dE/T)
    if dE <= 0:
        return True
    elif uni <= r:
        return True
    else:
        return False

@cython.cdivision(True)
@cython.boundscheck(False)
@cython.wraparound(False)
cdef void step(cnp.int32_t[::1] spins, cnp.int32_t[:, :] neigh, double T, int new_val, int site, npy_longdouble uni):
    """крутим 1 спин"""

        
    cdef int q, L2, dE
    
    q = 4
    L2 = spins.shape[0]
    
    dE = calc_dE_2(site, new_val, spins, neigh)
    
    if mc_choice(dE, T, uni):
        spins[site] = new_val
        
    
@cython.cdivision(True)
@cython.boundscheck(False)
@cython.wraparound(False)
def mc_step(cnp.int32_t[::1] spins,
            cnp.int32_t[:, :] neighbors,
            double T):
    """perform L*L flips for 1 MC step"""
    
    cdef npy_intp num_steps = spins.shape[0]
    cdef cnp.ndarray[double,
                ndim=1,
                negative_indices=False,
                mode='c'] unis = np.random.uniform(size=num_steps)
    cdef cnp.ndarray[npy_intp,
                ndim=1,
                negative_indices=False,
                mode='c'] new_vals = np.random.randint(4, size=num_steps)
    cdef cnp.ndarray[npy_intp,
                ndim=1,
                negative_indices=False,
                mode='c'] sites = np.random.randint(num_steps, size=num_steps)
    
    for _ in range(num_steps):
        step(spins, neighbors, T, new_vals[_], sites[_], unis[_])

In [137]:
wr = RndmWrapper(123)
%timeit [wr.py_uniform() for _ in range(100**2)]

1.44 ms ± 79.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [138]:
from numpy.random import PCG64, Generator
bitgen = PCG64(123)
rndm = Generator(bitgen)
%timeit [rndm.uniform() for _ in range(100**2)]

30.3 ms ± 5.7 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [139]:
np.random.seed(123)
%timeit np.random.uniform(size=100**2)

160 µs ± 35.3 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


## Run

In [33]:
for seed in range(1,9):
    
    global L, q, J
    L = 16
    q = 4      # components
    J = 1      # interaction energy

    N_avg = 5000
    N_mc = 10
    Relax = 1000

    np.random.seed(seed)
    
    tc = 1/(np.log(1+4**0.5)) # 0.9102392266268373
    t_ = np.array([0.001, 0.002, 0.005, 0.01, 0.05, 0.1])
    t_low = np.round(-t_*tc+tc, 3)  #low
    t_high = np.round(t_*tc+tc, 3)
    t = np.concatenate((t_low, t_high), axis=None)
    t.sort()
        
    df_e,df_m =[pd.DataFrame() for i in range(2)]
    st = time.time()
    for ind,T in enumerate(t):
        e,m = model_p4(T,N_avg,N_mc,Relax)
        df_e.insert(ind,T,e, True)
        df_m.insert(ind,T,m, True)
    title = 'potts4_L'+str(L)+'_avg'+str(N_avg)+'_mc'+str(N_mc)+'_relax'+str(Relax)+'mc_'
    df_e.to_csv('export/e_'+title+'seed'+str(seed)+'.csv', index = None, header=True)
    df_m.to_csv('export/m_'+title+'seed'+str(seed)+'.csv', index = None, header=True)
    print('im done in ',time.time()-st)

im done in  38.281044006347656
im done in  35.33965611457825
im done in  31.986581087112427
im done in  34.4723846912384
im done in  32.94367289543152
im done in  33.61117100715637
im done in  31.43293809890747
im done in  31.456133127212524


In [143]:
global L, q, J
L = 100
q = 4      # components
J = 1      # interaction energy
np.random.seed(1)
s = gen_state(); nei = get_neigh()

%timeit [mc_step(s, nei, 1) for _ in range(10)]

6.65 ms ± 130 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [142]:
global L, q, J
L = 100
q = 4      # components
J = 1      # interaction energy
np.random.seed(1)
s = gen_state(); nei = get_neigh()
print(calc_e(s))

for _ in range(10):
    mc_step(s, nei, 0.1)
print(calc_e(s))

-4959
-14188


In [ ]:
# py v1 - 1.49 s ± 97.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)    my origin
# cy v0 - 2.05 s ± 47.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)    tabulate neigh + cythonize(no random)
# cy v1 - 379 ms ± 11.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)    gen np.random array
# cy v2 - 344 ms ± 14.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)    cythonize gen random array
# cy v3 - 5.05 ms ± 219 µs per loop (mean ± std. dev. of 7 runs, 100 loops each) pure cython random generator !BAD!
# cy v3 - 27.1 ms ± 8.12 ms per loop (mean ± std. dev. of 7 runs, 10 loops each) Ev-Br RNG
# cy v4 - 6.65 ms ± 130 µs per loop (mean ± std. dev. of 7 runs, 100 loops each) gen all randoms, pass 1 by 1